### Importing and setting the Client ID and Secret for using the Spotify API

In [ ]:
import requests

CLIENT_ID = 'f0eb909d5e5149848a7a302e799a9e2b'
CLIENT_SECRET = 'e5140f6cdc874631afe2b18b4dfc61db'

### Setting the Auth URL and token for the Spotify API

In [ ]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

### Writing out the initial column names of the csv file

In [ ]:
f1 = open("results.csv", 'a')
factors = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
for f in factors:
    if f == "tempo":
        f1.write(f + '\n')
    else:
        f1.write(f + ',')

### Fetching the song data based on its track id and writing it to the results.csv

In [ ]:
data_path = os.path.join('spotify-processed-data.csv')
df = pd.read_csv(data_path, delimiter = ',')
BASE_URL = 'https://api.spotify.com/v1/'
for url in df['URL']:
    track_id = url[:31]
    response = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    response = response.json()
    if 'error' in response.keys():
        if response['error']['status'] == 404:
            print('track_id status not found', track_id)
            continue
        if response['error']['status'] == 400:
            print('track_id request invalid', track_id)
            continue
        elif response['error']['status'] == 429:
            print("Received 429 error from server")
            time.sleep(900)
            r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

    f1.write(track_id + ',')
    for f in factors:
        if f == "tempo":
            f1.write(str(response[f]) + '\n')
        else:
            f1.write(str(response[f]) + ','